In [ ]:
#_________________________IMPORT_________________________

# Import necessary libraries
import pandas as pd
import os
import re

#_________________________SPECIAL EXTRACTION_________________________

# Function to extract authors from a string using regex for group 2
def extract_authors_with_regex(author_str):
    if isinstance(author_str, str):
        matches = re.findall(r"'([^']+)'", author_str)
        return "; ".join(matches) if matches else None
    return None

# Function to extract the first researcher from a string for group 6
def get_first_researcher(researchers_str):
    if isinstance(researchers_str, str):
        return researchers_str.split(";")[0].strip()
    return None

#_________________________MAPPING FORMAT_________________________

# File paths and group mapping
file_map = {
    "group1.csv": {
        "ProjectTitle": "title",
        "ProjectRDC": "Agency",
        "OutputTitle": "title",
        "Abstract": "abstract",
        "ProjectPI": "project_pi",
        "OutputYear": "year",
        "DOI": "doi"
    },
    "group2.csv": {
        "ProjectTitle": "title",
        "OutputTitle": "title",
        "Abstract": "abstract",
        "ProjectPI": "researcher",
        "Authors": "authors",
        "ProjectRDC": "location",
        "OutputYear": "year",
        "DOI": "doi"
    },
    "group3.csv": {
        "ProjectTitle": "Title",
        "OutputTitle": "Title",
        "Abstract": "Abstract",
        "ProjectPI": "PI",
        "Authors": "author",
        "OutputYear": "publication_year",
        "ProjectRDC": "RDC",
        "DOI": "doi"
    },
    "group4.csv": {
        "OutputTitle": "title",
        "Abstract": "abstract",
        "ProjectPI": "researcher",
        "OutputYear": "year",
    },
    "group5.csv": {
        "ProjectTitle": "title_clean",
        "OutputTitle": "title",
        "ProjectPI": "pi",
        "OutputYear": "year",
        "DOI": "doi"
    },
    "group6.csv": {
        "ProjectTitle": "Title",
        "OutputTitle": "Title",
        "Abstract": "Abstract",
        "Authors": "Researchers",
        "OutputYear": "Year",
        "OutputBiblio": "Keywords",
        "DOI": "DOI",
        "ProjectRDC": "RDC",
    },
    "group7.csv": {
        "OutputTitle": "title",
        "Abstract": "abstract",
    },
    "group8.csv": {
        "ProjID": "ProjectID",
        "ProjectTitle": "ProjectTitle",
        "ProjectRDC": "ProjectRDC",
        "ProjectPI": "ProjectPI",
        "OutputTitle": "OutputTitle",
        "OutputBiblio": "OutputBiblio",
        "OutputYear": "OutputYear",
        "DOI": "DOI",
        "Abstract": "Abstract",
        "Authors": "Authors"
    }
}
# Final desired columns
final_columns = [
    "ProjID", "ProjectTitle", "ProjectRDC", "ProjectPI", "Authors",
    "OutputTitle", "OutputBiblio", "OutputYear", "DOI", "Abstract", "source_file"
]

#_________________________MAIN EXECUTION FUNCTION_________________________

def main():
    # Directory where files are stored
    data_dir = "./"

    # List to hold standardized DataFrames
    standardized_dfs = []
#_________________________MAPPING & MERGING_________________________

# Load, rename, and align each file 
    for file, mapping in file_map.items():
        path = os.path.join(data_dir, file)
        try:
            df = pd.read_excel(path) if file.endswith(".xlsx") else pd.read_csv(path)

            # Normalize all column names for safer matching
            df.columns = [col.strip().lower() for col in df.columns]
            normalized_mapping = {k: v.lower().strip() for k, v in mapping.items()}

            df_renamed = pd.DataFrame()

            for final_col in final_columns:
                src_col = normalized_mapping.get(final_col)
                if src_col and src_col in df.columns:
                    df_renamed[final_col] = df[src_col]
                else:
                    df_renamed[final_col] = None

            # Apply author regex parsing for Group 2
            if file == "group2.csv" and "Authors" in df_renamed.columns:
                df_renamed["Authors"] = df_renamed["Authors"].apply(extract_authors_with_regex)

            # Apply PI extraction for Group 6
            if file == "group6.csv" and "researchers" in df.columns:
                df_renamed["ProjectPI"] = df["researchers"].apply(get_first_researcher)

            df_renamed["source_file"] = file
            standardized_dfs.append(df_renamed)

        except FileNotFoundError:
            print(f"Missing file: {file} — skipping.")
        except Exception as e:
            print(f"Error processing {file}: {e}")


    # Filter out empty or all-NA DataFrames
    standardized_dfs = [
        df for df in standardized_dfs
        if not df.empty and df.dropna(how="all").shape[0] > 0
    ]
    # Combine all
    combined_final_df = pd.concat(standardized_dfs, ignore_index=True)

    #  Number of rows after merging
    before_doi = len(combined_final_df)
    print(f"Initial merged rows: {before_doi}")
#_________________________DISCARDING_________________________

    # Drop rows missing DOI
    combined_final_df.columns = [col.strip().lower() for col in combined_final_df.columns]
    if 'doi' in combined_final_df.columns:
        combined_final_df = combined_final_df[
            ~combined_final_df['doi'].isna() & (combined_final_df['doi'].str.strip() != "")
        ]
    else:
        print("Warning: 'DOI' column not found. No filtering applied.")

    # Number of rows after discarding missing DOIs
    after_doi = len(combined_final_df)
    print(f"Discarded due to missing DOI: {before_doi - after_doi}")
#_________________________DEDUPLICATION_________________________

    # Drop duplicate DOIs, keeping the first occurrence
    combined_final_df['doi'] = combined_final_df['doi'].str.lower().str.strip()
    combined_final_df = combined_final_df.drop_duplicates(subset='doi', keep='first')

    # Number of rows after dropping duplicates
    after_dedup = len(combined_final_df)
    print(f"Removed duplicates by DOI: {after_doi - after_dedup}")
    print(f"Final row count: {after_dedup}")
#_________________________SAVE TO CSV_________________________
    # Save 
    combined_final_df.to_csv("Final_ResearchOutputs_Cleaned.csv", index=False)
    print("Saved: Final_ResearchOutputs_Cleaned.csv")

#_________________________RUN MAIN_________________________

if __name__ == "__main__":
    main()


C:\Users\liamr\AppData\Local\Temp\ipykernel_34740\3778030204.py:152: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_final_df = pd.concat(standardized_dfs, ignore_index=True)


Initial merged rows: 39644
Discarded due to missing DOI: 21667
Removed duplicates by DOI: 923
Final row count: 17054
Saved: Final_ResearchOutputs_Cleaned.csv
